# <center>Result_Scraper</center>

## Steps: 
1. Open Meeting Overview page
2. Detect how many races are listed
3. Collect each race’s URL
4. Visit each race page one by one
5.  Scrape:
    1. horse names
    2. barrier
    3. weight
    4. jockey
    5. time
    6. margin

In [1]:
import pandas as pd 
import numpy as np

import requests
from bs4 import BeautifulSoup as bs
from bs4 import NavigableString

### Main Webpage

In [2]:
# Load the webpage
r = requests.get('https://loveracing.nz/RaceInfo/54877/Meeting-Overview.aspx')

# covert to a beautiful soup object
webpage = bs(r.content)

print(webpage.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <!--<![endif]-->
 <head>
  <title>
   Race Meeting for WELLINGTON RC at TRENTHAM on 17 JAN 2026 | LOVERACING.NZ
  </title>
  <meta content="" id="keywords" name="keywords"/>
  <meta content="Fields for WELLINGTON RC on 17 JAN 2026" id="description" name="description"/>
  <!-- INCLUDES CONTROL -->
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="/Common/Images/v-3.191/favicon.png" rel="shortcut icon"/>
  <link href="/Common/Style/v-3.191/bootstrap.min.css" rel="stylesheet"/>
  <link href="/Common/Style/v-3.191/bootstrap-grid.min.css" rel="stylesheet"/>
  <link href="/Common/Style/v-3.191/bootstrap-reboot.min.css" rel="stylesheet"/>
  <link href="/Style/v-3.191/web.css" rel="stylesheet"/>
  <link data-noprefix="" href="//fonts.googleapis.com/css?family=Open+Sans:400,600,400italic,600italic" rel="stylesheet" type="text/css"/>
  <link href="/Common/UI/fancybox/v-3.191/jquery.fancybox.cs

In [3]:
race_name = webpage.find_all("div", class_="racename")
race_name

[<div class="racename"><a href="/RaceInfo/54877/1/Race-Detail.aspx" title="View Race 1">ROTHLEY MAIDEN 1400 (FOR MAIDENS AS AT MIDNIGHT 31 OCTOBER)</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/2/Race-Detail.aspx" title="View Race 2">GEE &amp;  HICKTON FUNERAL DIRECTORS SPRINT</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/3/Race-Detail.aspx" title="View Race 3">AZTECH ENGINEERING LTD SPRINT</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/4/Race-Detail.aspx" title="View Race 4">HAPPY HIRE 2100</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/5/Race-Detail.aspx" title="View Race 5">HARDY TRADE SUPPLY CO 2200</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/6/Race-Detail.aspx" title="View Race 6">TOTARA LODGE TRENTHAM STAKES</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/7/Race-Detail.aspx" title="View Race 7">NZB DESERT GOLD STAKES</a></div>,
 <div class="racename"><a href="/RaceInfo/54877/8/Race-Detail.aspx" title="Vie

In [4]:
links = []

for div in race_name:
    a_tag = div.find("a")
    if a_tag and a_tag.has_attr("href"):
        links.append(a_tag["href"])

print(links)

['/RaceInfo/54877/1/Race-Detail.aspx', '/RaceInfo/54877/2/Race-Detail.aspx', '/RaceInfo/54877/3/Race-Detail.aspx', '/RaceInfo/54877/4/Race-Detail.aspx', '/RaceInfo/54877/5/Race-Detail.aspx', '/RaceInfo/54877/6/Race-Detail.aspx', '/RaceInfo/54877/7/Race-Detail.aspx', '/RaceInfo/54877/8/Race-Detail.aspx', '/RaceInfo/54877/9/Race-Detail.aspx']


### Table page links

In [5]:
base_url = 'https://loveracing.nz'

i = 0 # counter variable
for link in links: 
    i = i+1
    print(i,'. ',base_url + link)

1 .  https://loveracing.nz/RaceInfo/54877/1/Race-Detail.aspx
2 .  https://loveracing.nz/RaceInfo/54877/2/Race-Detail.aspx
3 .  https://loveracing.nz/RaceInfo/54877/3/Race-Detail.aspx
4 .  https://loveracing.nz/RaceInfo/54877/4/Race-Detail.aspx
5 .  https://loveracing.nz/RaceInfo/54877/5/Race-Detail.aspx
6 .  https://loveracing.nz/RaceInfo/54877/6/Race-Detail.aspx
7 .  https://loveracing.nz/RaceInfo/54877/7/Race-Detail.aspx
8 .  https://loveracing.nz/RaceInfo/54877/8/Race-Detail.aspx
9 .  https://loveracing.nz/RaceInfo/54877/9/Race-Detail.aspx


### Testing scraper workflow in one table

In [6]:
race1 = base_url + links[0]
print(race1)

r = requests.get(race1)
web1 = bs(r.content)
print(web1)

https://loveracing.nz/RaceInfo/54877/1/Race-Detail.aspx
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]--><!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]--><!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en"> <!--<![endif]-->
<head>
<title>Race 1 ROTHLEY MAIDEN 1400 (FOR MAIDENS AS AT MIDNIGHT 31 OCTOBER) at WELLINGTON RC on 17 JAN 2026 | LOVERACING.NZ</title>
<meta content="" id="keywords" name="keywords"/>
<meta content="Results for ROTHLEY MAIDEN 1400 (FOR MAIDENS AS AT MIDNIGHT 31 OCTOBER) on 17 JAN 2026" id="description" name="description"/>
<!-- INCLUDES CONTROL -->
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/Common/Images/v-3.191/favicon.png" rel="shortcut icon"/>
<link href="/Common/Style/v-3.191/bootstrap.min.css" rel="stylesheet"/>
<link href="/Common/Style/v-3.191/bootstrap-grid.m

In [7]:
data = {}
# positions
positions = web1.find_all("div", class_="column race-number placing")
data['positions'] = [int(p.get_text(strip=True)) for p in positions[1:]]

In [8]:
#jokey 
jockeies = web1.find_all("span", class_="jockey")
data['jokey'] = [p.get_text(strip=True) for p in jockeies]

In [9]:
# horse 
horses = web1.find_all("div", class_="column horse")

horse_names = []

for horse_div in horses:
    horse_name = next(
        (
            t.strip()
            for t in horse_div.contents
            if isinstance(t, NavigableString)
            and t.strip()
            and not t.strip().startswith('=')
        ),
        None  
    )

    if horse_name:   # only add valid names
        horse_names.append(horse_name)

data['horse'] = horse_names


In [10]:
# bar 
bars = web1.find_all("div", class_="column col3")
b = [int(p.get_text(strip=True)) for p in bars[1:]]
data['bar'] = b

In [11]:
# weight carried by horse
weights = web1.find_all("div", class_='column col5')
b = [float(p.get_text(strip=True)) for p in weights[1:]]
data['weight'] = b

In [12]:
# time
time = web1.find_all("div", class_='column col9')
b = [p.get_text(strip=True) for p in time[1:]]
data['time'] = b

In [13]:
# margin
margin = web1.find_all("div", class_='column col7')
b = [p.get_text(strip=True) for p in margin[1:]]
data['margin'] = b

In [14]:
print(data)
print('\n',data.keys())

{'positions': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'jokey': ['Samantha Collett', 'Matthew Cameron', 'Kelly Myers', 'Kate Hercock', 'Craig Grylls', 'Matt Cartwright', 'Joe Doyle', 'Rory Hutchings', 'Madan Singh', 'George Rooke', 'Jim Chung (A)'], 'horse': ['Hoorah Henry', "I'll Be Home Soon", 'Wyworry', 'Elegant Elle', 'Wild At Heart', 'Furiosa', 'Belles Boom', 'Dancing Shadow', 'Teresa Mac', 'Clear Conscience (AUS)', 'Derrick'], 'bar': [6, 2, 1, 11, 9, 10, 7, 8, 5, 4, 3], 'weight': [58.5, 58.5, 58.5, 54.0, 56.5, 55.0, 57.0, 57.0, 55.0, 56.5, 57.0], 'time': ['1.26.31', '1.26.55', '1.26.63', '1.26.64', '1.26.81', '1.26.85', '1.27.00', '1.27.08', '1.27.15', '1.27.22', '1.27.64'], 'margin': ['', '1.37L', '1.79L', '1.86L', '2.84L', '3.06L', '3.92L', '4.38L', '4.76L', '5.18L', '7.55L']}

 dict_keys(['positions', 'jokey', 'horse', 'bar', 'weight', 'time', 'margin'])


In [15]:
d = pd.DataFrame(data)
d

,positions,jokey,horse,bar,weight,time,margin
0,1,Samantha Collett,Hoorah Henry,6,58.5,1.26.31,
1,2,Matthew Cameron,I'll Be Home Soon,2,58.5,1.26.55,1.37L
2,3,Kelly Myers,Wyworry,1,58.5,1.26.63,1.79L
3,4,Kate Hercock,Elegant Elle,11,54.0,1.26.64,1.86L
4,5,Craig Grylls,Wild At Heart,9,56.5,1.26.81,2.84L
5,6,Matt Cartwright,Furiosa,10,55.0,1.26.85,3.06L
6,7,Joe Doyle,Belles Boom,7,57.0,1.27.00,3.92L
7,8,Rory Hutchings,Dancing Shadow,8,57.0,1.27.08,4.38L
8,9,Madan Singh,Teresa Mac,5,55.0,1.27.15,4.76L
9,10,George Rooke,Clear Conscience (AUS),4,56.5,1.27.22,5.18L


### Scraping all the tables

In [19]:
all_data = []  # to store dataframes for each race

for i, link in enumerate(links, start=1):  # start=1 so race numbers start from 1
    race_url = base_url + link
    print(f"Scraping race {i}: {race_url}")
    
    try:
        r = requests.get(race_url)
        r.raise_for_status()  # raise exception for bad status codes
        web = bs(r.content, 'html.parser')
        
        # Find all columns
        positions = web.find_all("div", class_="column race-number placing")[1:]
        jockeys = web.find_all("span", class_="jockey")
        horses_divs = web.find_all("div", class_="column horse")
        bars = web.find_all("div", class_="column col3")[1:]
        weights = web.find_all("div", class_='column col5')[1:]
        times = web.find_all("div", class_='column col9')[1:]
        margins = web.find_all("div", class_='column col7')[1:]
        
        # Loop through rows
        race_rows = []
        for idx in range(len(positions)):
            try:
                pos = int(positions[idx].get_text(strip=True))
                jockey = jockeys[idx].get_text(strip=True)
                
                # horse name extraction
                horse_div = horses_divs[idx]
                horse_name = next(
                    (
                        t.strip()
                        for t in horse_div.contents
                        if isinstance(t, NavigableString)
                        and t.strip()
                        and not t.strip().startswith('=')
                    ),
                    None
                )
                if not horse_name:  # skip row if horse name not found
                    raise ValueError("Horse name missing")
                
                bar = int(bars[idx].get_text(strip=True))
                weight = float(weights[idx].get_text(strip=True))
                time = times[idx].get_text(strip=True)
                margin = margins[idx].get_text(strip=True)
                
                race_rows.append({
                    'positions': pos,
                    'jockey': jockey,
                    'horse': horse_name,
                    'bar': bar,
                    'weight': weight,
                    'time': time,
                    'margin': margin,
                    'race_number': i
                })
                
            except Exception as row_err:
                print(f"Skipping row {idx+1} in race {i}: {row_err}")
        
        # Create dataframe for this race
        if race_rows:
            df_race = pd.DataFrame(race_rows)
            all_data.append(df_race)
        
    except Exception as e:
        print(f"Error scraping race {i} ({race_url}): {e}")

# combine all races into a single dataframe
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df.head())
else:
    print("No data scraped.")

Scraping race 1: https://loveracing.nz/RaceInfo/54877/1/Race-Detail.aspx
Skipping row 1 in race 1: Horse name missing
Scraping race 2: https://loveracing.nz/RaceInfo/54877/2/Race-Detail.aspx
Skipping row 1 in race 2: Horse name missing
Skipping row 5 in race 2: invalid literal for int() with base 10: 'SCR'
Scraping race 3: https://loveracing.nz/RaceInfo/54877/3/Race-Detail.aspx
Skipping row 1 in race 3: Horse name missing
Scraping race 4: https://loveracing.nz/RaceInfo/54877/4/Race-Detail.aspx
Skipping row 1 in race 4: Horse name missing
Skipping row 17 in race 4: invalid literal for int() with base 10: ''
Skipping row 18 in race 4: invalid literal for int() with base 10: ''
Skipping row 19 in race 4: invalid literal for int() with base 10: ''
Skipping row 20 in race 4: invalid literal for int() with base 10: ''
Skipping row 21 in race 4: invalid literal for int() with base 10: ''
Skipping row 22 in race 4: invalid literal for int() with base 10: ''
Scraping race 5: https://loveracing.

In [21]:
final_df.head(50)

,positions,jockey,horse,bar,weight,time,margin,race_number
0,2,Matthew Cameron,Hoorah Henry,2,58.5,1.26.55,1.37L,1
1,3,Kelly Myers,I'll Be Home Soon,1,58.5,1.26.63,1.79L,1
2,4,Kate Hercock,Wyworry,11,54.0,1.26.64,1.86L,1
3,5,Craig Grylls,Elegant Elle,9,56.5,1.26.81,2.84L,1
4,6,Matt Cartwright,Wild At Heart,10,55.0,1.26.85,3.06L,1
5,7,Joe Doyle,Furiosa,7,57.0,1.27.00,3.92L,1
6,8,Rory Hutchings,Belles Boom,8,57.0,1.27.08,4.38L,1
7,9,Madan Singh,Dancing Shadow,5,55.0,1.27.15,4.76L,1
8,10,George Rooke,Teresa Mac,4,56.5,1.27.22,5.18L,1
9,11,Jim Chung (A),Clear Conscience (AUS),3,57.0,1.27.64,7.55L,1


In [24]:
final_df.shape

(98, 8)

In [23]:
final_df.to_csv('horse_runs.csv', index=False)
